<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(nftables_rate_limit_black-box_attackers.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

### NFTables Rate Limiting Black-box attackers  

### Lab Objectives:   
- Drop any source IP address, sending more than 50 packets per minute to our API endpoint  
- Prent our API endpoint from seeing or processing any additional packets for the suspicious source host   
- Viewing the IPs that are blocked  
- Working with named sets to track the source IPs that are blocked  
- Leveraging meter maps with rate-limiting by source  
- Monitor who is blocked   


Now that we know how threat actors can target our API endpoints, let us reduce the risk associated with having our API endpoints available.  For this, we will use our firewall. In our case, we are using **nftables**: https://netfilter.org/projects/nftables/manpage.html   

- Additional References: https://docs.redhat.com/en/documentation/red_hat_enterprise_linux/7/html/security_guide/sec-using_nftables_to_limit_the_amount_of_connections    

https://wiki.nftables.org/wiki-nftables/index.php/Meters   

#### Everything below must be done in your WSL or Linux terminal.  
#### We could run these commands in this notebook but choose not to. 

#### Revisiting the logs  
If we look at the log file we created in the previous lab, we would find over 2000 rows. This correlates with the number of queries we made. 

$ **wc --lines /tmp/mlflow_serve.log**  
2003 /tmp/mlflow_serve.log

If we look at the difference between the start and end timelines we see these 2000+ lines have a difference of just over 12 minutes.   

$ head --lines 2 /tmp/mlflow_serve.log ; tail --lines 1 /tmp/mlflow_serve.log
[2025-06-16 14:20:13] INFO:     127.0.0.1:40892 - "GET /version HTTP/1.1" 200 OK
[2025-06-16 14:20:14] INFO:     10.0.0.1:0 - "POST /invocations HTTP/1.1" 200 OK
[2025-06-16 14:32:56] INFO:     10.0.0.1:0 - "POST /invocations HTTP/1.1" 200 OK

Note: My timestamp and yours will be different as we are running the labs at different times.   

## Please pay attention to the escape sequence "\" when using bash 

## Creating the *nft* table

- Let us start with flushing the nfttable. This ensures we are starting off from a fresh setup    
**$ sudo nft flush ruleset**   

- Verify now that there is nothing in the table       
**$ sudo nft --numeric list ruleset**    

Assuming our table is now empty, let us create our protection   
We could use nft **create** command. Let's use **nft add**   instead
**nft add** will add the table *if* it does not already exist. If we use **nft create** and the table exists, it will throw an error. Let us try to avoid any possible errors.  


$ **sudo nft add table ip INFERENCE_EP { comment \" ML_INFERENCE_ENDPOINT PROTECTION \" \; }**    

### What is being done above?       
- sudo: We need root level privilege to use nftable. Hence we are elevating our privileges    
- nft: Invokes the nft tool. We could have done all of this in the nft *interactive* environment, by using *sudo nft --interactive*. We will however, stick to writing our commands out on the command line  
- add: We use add here but could have used create. As mentioned above, add will not return an error if the table exists already. However, create will return an error. We are just ensuring we avoid any errors even though we know the table should not be existing because we arleady *flush* the ruleset   
- ip: This says we are creating a table that supports IPv4.      
- INFERENCE_EP: The name of our table   
- The rest of the items within the quotes '' is just the comment to ensure we are being more descriptive. 


### Verify the creation of our table  
**$ sudo nft --numeric list ruleset**      
table ip INFERENCE_EP {     
        comment " ML_INFERENCE_ENDPOINT PROTECTION "    
}     

Now that we were able to validate that we have successfully created our table, let us move ahead to creating our chain. This is the container for our rules. 

## Creating a Chain: Container for our rules   
$ **sudo nft add chain ip INFERENCE_EP input { type filter hook input priority 0 \; policy accept \; comment \"ML_INFERENCE_ENDPOINT RULES CONTAINER \" \; }**    

### What is being done above?       
Let us only focus on the new items   
- chain: This specifies that we are setting up a container for our rules   
- input: This chain will focus on traffic coming into the firewall   
- type filter: We are using this for packet filtering rather than nat or route.   
- hook input: Traffic entering the firewall   
- priority 0: The importance of the rule. Lower numbers are more important    
- policy accept: This chain will be accepting traffic   


### Verify the creation of chain   
$ **sudo nft --numeric list chains ip**    
table ip INFERENCE_EP {    
        chain input {   
                comment "ML_INFERENCE_ENDPOINT RULES CONTAINER "   
                type filter hook input priority 0; policy accept;    
        }    
}    


### Verify the table has been updated   
$ **sudo nft --handle --numeric list ruleset**  
table ip INFERENCE_EP { # handle 26   
        comment " ML_INFERENCE_ENDPOINT PROTECTION "   
        chain input { # handle 1    
                comment "ML_INFERENCE_ENDPOINT RULES CONTAINER "    
                type filter hook input priority 0; policy accept;    
        }    
}    

Let us now move to creating a named set


## Creating a named set 
Sets come in two forms, anonymous and named. We will work with named as it allows us to update the sets in the future if needed.   

$ **sudo nft add set ip INFERENCE_EP suspicious_api_access { type ipv4_addr \; flags dynamic, timeout \; timeout 5m \; comment \" SUSPICIOUS IPS \" \; }**     

### What is being done above?       
We will only focus on the new items. 
- set: this specifies that we will be using a named set 
- suspicious_api_access: the name of our set   
- type ipv4_addr: We will focus on IPv4 addresses   
- flags dynamic timeout: This is a dynamic entry that will timeout       
- timeout 5m: Remove the entry from the list after 5 minutes      


### Verify the creation of our set  
**$ sudo nft --numeric list sets**    
table ip INFERENCE_EP {    
        set suspicious_api_access {    
                type ipv4_addr    
                flags dynamic,timeout    
                timeout 300s    
                comment " SUSPICIOUS IPS "    
        }    
}    


### Verify the table has been updated   
**$ sudo nft --handle --numeric list ruleset**   
table ip INFERENCE_EP { # handle 26   
        comment " ML_INFERENCE_ENDPOINT PROTECTION "    
        set suspicious_api_access { # handle 2    
                type ipv4_addr    
                flags dynamic,timeout    
                timeout 300s    
                comment " SUSPICIOUS IPS "    
        }    

        chain input { # handle 1   
                comment "ML_INFERENCE_ENDPOINT RULES CONTAINER "    
                type filter hook input priority 0; policy accept;    
        }    
}    

### Add our first rule

Above should return an empty set. Let us update this.   
$ **sudo nft add rule ip INFERENCE_EP input ip protocol tcp tcp dport 5000 ct state new, untracked limit rate over 50/minute add @suspicious_api_access { ip saddr } accept comment \" ML_INFERENCE_ENDPOINT incoming connections \"**   

1. run: $ *sudo nft --handle --numeric list ruleset* to get the rule index number    
2. With the rule index number in place run $ *sudo nft delete rule inet ML_AI_API_PROTECTION input handle PUT_YOUR_RULE_INDEX_NUMBER_HERE_FROM_STEP_1*
3. Alternatively restart the entire process  


### What is being done above?       
Once again, let us only focus on the new items   
- rule: We wish to create a rule   
- ip protocol tcp: The focus is on the TCP protocol    
- tcp dport 5000: We are restricting our traffic to particularly tcp destination port 5000    
- ct state new: Track new connections    
- untracked limit rate over 50/minute add @suspicious_api_access { ip saddr } accept: for IPs under the threshold of 50 packets per minute ignore that. Anything over add to the suspicious_api_access set    


### Verify the table has been updated  
$ **sudo nft --handle --numeric list ruleset**    
table ip INFERENCE_EP { # handle 26   
        comment " ML_INFERENCE_ENDPOINT PROTECTION "    
        set suspicious_api_access { # handle 2    
                type ipv4_addr    
                size 65535   
                flags dynamic,timeout   
                timeout 300s    
                comment " SUSPICIOUS IPS "   
        }    

        chain input { # handle 1   
                comment "ML_INFERENCE_ENDPOINT RULES CONTAINER "    
                type filter hook input priority 0; policy accept;    
                tcp dport 5000 ct state 0x8,0x40 limit rate over 50/minute add @suspicious_api_access { ip saddr } accept comment " ML_INFERENCE_ENDPOINT incoming connections " # handle 3    
        }    
}    




## Wrap this up!    
$ **sudo nft add rule ip INFERENCE_EP input ip saddr @suspicious_api_access tcp dport 5000 log prefix \" SUSPICIOUS CONNECTIONS DROPPED \" counter drop**


**note** if for some strange you create a wrong rule, you can delete the rule as follows:   
1. run: $ *sudo nft --handle --numeric list ruleset* to get the rule index number    
2. With the rule index number in place run $ *sudo nft delete rule inet ML_AI_API_PROTECTION input handle PUT_YOUR_RULE_INDEX_NUMBER_HERE_FROM_STEP_1*
3. Alternatively restart the entire process  


### What is being done above?       
Once again, let us only focus on the new items   
- ip saddr @suspicious_api_access: Any IPs in this set should be dropped       
- tcp dport 5000 drop: We will be dropping traffic on TCP destination port 5000
- log prefix: Allows us to leave a comment in the log    
- counter: Return the counts of the packets and bytes    


### Verify the table has been updated  
$ sudo nft --handle --numeric list ruleset
table ip INFERENCE_EP { # handle 26    
        comment " ML_INFERENCE_ENDPOINT PROTECTION "    
        set suspicious_api_access { # handle 2    
                type ipv4_addr    
                size 65535    
                flags dynamic,timeout   
                timeout 300s    
                comment " SUSPICIOUS IPS "    
        }    

        chain input { # handle 1    
                comment "ML_INFERENCE_ENDPOINT RULES CONTAINER "   
                type filter hook input priority 0; policy accept;     
                tcp dport 5000 ct state 0x8,0x40 limit rate over 50/minute add @suspicious_api_access { ip saddr } accept comment " ML_INFERENCE_ENDPOINT incoming connections " # handle 3    
                ip saddr @suspicious_api_access tcp dport 5000 log prefix " SUSPICIOUS CONNECTIONS DROPPED " counter packets 0 bytes 0 drop # handle 4     
        }    
}    

###
Run the lab **knockoff-nets_v2_rate_limited_nftables.ipynb**. You should see the some packets come in on mlflow as well as via the *tcpdump* you should have running. The rest will be be blocked by the firewall.    
As your attack is running you should see something similar to:   


$ **sudo nft --handle --numeric list ruleset**   
table ip INFERENCE_EP { # handle 26   
        comment " ML_INFERENCE_ENDPOINT PROTECTION "    
        set suspicious_api_access { # handle 2    
                type ipv4_addr   
                size 65535   
                flags dynamic,timeout    
                timeout 300s    
                comment " SUSPICIOUS IPS "   
                elements = { **127.0.0.1 timeout 300s expires 298s** }   
        }   

        chain input { # handle 1    
                comment "ML_INFERENCE_ENDPOINT RULES CONTAINER "    
                type filter hook input priority 0; policy accept;    
                tcp dport 5000 ct state 0x8,0x40 limit rate over 50/minute add @suspicious_api_access { ip saddr } accept comment " ML_INFERENCE_ENDPOINT incoming connections " # handle 3    
                ip saddr @suspicious_api_access tcp dport 5000 log prefix " SUSPICIOUS CONNECTIONS DROPPED " **counter packets 8 bytes 7964 drop # handle 4**    
        }    
}


You can also validate that information is in your logs  
First by using **dmesg**: $ **sudo dmesg --ctime**   
then by $ **cat /var/log/syslog | grep SUSPICIOUS**   

### Takeaways  
- We were able to use the firewall, in this case *nftables* to mitigate the threat associated with query our models to create a copy   
- We learnt how to use *nftables* command to create tables, chains, set, rules, meters, etc.   
- We learned that this threat can be contained just like we would address other infrastructure cybersecurity challenges   

### Optional to help with testing your firewalls:   
Testing our rule
Open or split your current window into 4 different screens.


let's start with installing hping3 
$ sudo apt-install hping3


In a separate window run tcpdump
$sudo tcpdump -nni any port 5000


In a separate window run a necat listener
$ncat --verbose --listen --keep-open 5000


#### In another window make a connection to this netcat session to validate you can access the current listening session
$ ncat --verbose 127.0.0.1 5000


Assuming everything works as expected, you should have been able to make a successful connection from the client to the server. At the same time, you should see a successful 3-way handshake completed, which means your connection is now established.

Press CTRL+C to break out of the ncat client window


Now that we know that works, let's leverage HPing3 to generate some traffic

$ sudo hping3 --count 20 --baseport 9876 --destport 5000 --keep  --syn  --interface eth0 127.0.0.1 --faster


Now let's run HPing3 again this time increase the count parameter to a value of 100 or greater.
$ sudo hping3 --count 100 --baseport 9876 --destport 5000 --keep  --syn  --interface eth0 127.0.0.1 --faster
